In [ ]:
# If running in Google Colab, import files
try:
    import google.colab
    in_colab = True
except:
    in_colab = False

if in_colab:
    !git clone https://github.com/aiken516/one-pixel-attack-keras.git
    !mv -v one-pixel-attack-keras/* .
    !rm -rf one-pixel-attack-keras

# Python Libraries
%matplotlib inline
import pickle
import numpy as np
import pandas as pd
import matplotlib
from keras.datasets import cifar10
from keras import backend as K

# Custom Networks
from networks.lenet import LeNet
#from networks.pure_cnn import PureCnn
from networks.network_in_network import NetworkInNetwork
from networks.resnet import ResNet
from networks.densenet import DenseNet
from networks.wide_resnet import WideResNet
from networks.capsnet import CapsNet

# Helper functions
from differential_evolution import differential_evolution
import helper

import time

matplotlib.style.use('ggplot')
np.random.seed(100)

Cloning into 'one-pixel-attack-keras'...
remote: Enumerating objects: 385, done.
remote: Counting objects: 100% (45/45), done.
remote: Compressing objects: 100% (28/28), done.
remote: Total 385 (delta 22), reused 35 (delta 17), pack-reused 340
Receiving objects: 100% (385/385), 45.38 MiB | 19.62 MiB/s, done.
Resolving deltas: 100% (220/220), done.
renamed 'one-pixel-attack-keras/1_one_pixel_attack_cifar10.ipynb' -> './1_one_pixel_attack_cifar10.ipynb'
renamed 'one-pixel-attack-keras/2_one-pixel-attack-imagenet.ipynb' -> './2_one-pixel-attack-imagenet.ipynb'
renamed 'one-pixel-attack-keras/attack.py' -> './attack.py'
renamed 'one-pixel-attack-keras/CONTRIBUTING.md' -> './CONTRIBUTING.md'
renamed 'one-pixel-attack-keras/data' -> './data'
renamed 'one-pixel-attack-keras/differential_evolution.py' -> './differential_evolution.py'
renamed 'one-pixel-attack-keras/helper.py' -> './helper.py'
renamed 'one-pixel-attack-keras/images' -> './images'
renamed 'one-pixel-attack-keras/LICENSE' -> './L

## Load Dataset

For this attack, we will use the [Cifar10 dataset](https://www.cs.toronto.edu/~kriz/cifar.html) packaged by Keras. The task of the dataset is to correctly classify a 32x32 pixel image in 1 of 10 categories (e.g., bird, deer, truck).

The code below will load the Cifar10 dataset. Keras will need to download the dataset if it is not cached locally already.

In [ ]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

170498071/170498071 [==============================] - 10s 0us/step


In [ ]:
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

In [ ]:
def perturb_image(xs, img):
    # If this function is passed just one perturbation vector,
    # pack it in a list to keep the computation the same
    if xs.ndim < 2:
        xs = np.array([xs])

    # Copy the image n == len(xs) times so that we can
    # create n new perturbed images
    tile = [len(xs)] + [1]*(xs.ndim+1)
    imgs = np.tile(img, tile)

    # Make sure to floor the members of xs as int types
    xs = xs.astype(int)

    for x,img in zip(xs, imgs):
        # Split x into an array of 5-tuples (perturbation pixels)
        # i.e., [[x,y,r,g,b], ...]
        pixels = np.split(x, len(x) // 5)
        for pixel in pixels:
            # At each pixel's x,y position, assign its rgb value
            x_pos, y_pos, *rgb = pixel
            img[x_pos, y_pos] = rgb

    return imgs

In [ ]:
lenet = LeNet()
resnet = ResNet()

models = [lenet, resnet] #, resnet]

## Uncomment below to load more models to play with. Make sure the model files exist by training or downloading them.

# lenet = LeNet()
# pure_cnn = PureCnn()
# net_in_net = NetworkInNetwork()
# resnet = ResNet()
# densenet = DenseNet()
# wide_resnet = WideResNet()
# capsnet = CapsNet()

# models = [lenet, pure_cnn, net_in_net, resnet, densenet, wide_resnet, capsnet]

Successfully loaded lenet
Successfully loaded resnet


In [ ]:
def pixel_confidence(image_id):
    img_original = x_test[image_id].copy()
    target_class = y_test[image_id, 0]
    print("target_class: {0}".format(target_class))

    helper.plot_image(img_original)

    prior_confidence =  model.predict_one(img_original)[target_class]
    print("prior_confidence: {0}".format(prior_confidence))

    diff_list = [[0,0,0]]
    rgb = [0, 0, 0]

    array = np.empty((0, 32, 32, 3))

    for x in range(32):
        for y in range(32):
          for i in range(3):
              if img_original[x][y][i] >= 128:
                rgb[i] = 0
              else:
                rgb[i] = 255

          pixel = np.array([x, y,  rgb[0], rgb[1], rgb[2]])
          perturbed_image = perturb_image(pixel, img_original)

          array = np.append(array, np.array((perturbed_image)), axis=0)

    processed = model.color_process(array)
    result = model._model.predict(processed, batch_size=1024,verbose=0)

    for i in range(1024):
        after_confidence =  result[i][target_class]
        #print("after_confidence: {0}".format(after_confidence))

        diff_confidence = abs(after_confidence - prior_confidence)
        #print("diff_confidence: {0}".format(diff_confidence))

        if diff_confidence > 0:
            for diff_index in range(len(diff_list)):
                if diff_confidence >= diff_list[diff_index][1]:
                    diff_list.insert(diff_index, [i, diff_confidence])
                    if (len(diff_list) > 30):
                        diff_list.pop()
                    break

    return diff_list[0][0]

In [ ]:
def gradient_descent_momentum(image_id, pixel):
    img = x_test[image_id].copy()
    true_class = y_test[image_id, 0]

    x = pixel[0]
    y = pixel[1]
    r = pixel[2]
    g = pixel[3]
    b = pixel[4]

    rate = 5000
    momentum_rate = 0.99

    v_r = 0 #속도
    v_g = 0
    v_b = 0

    for i in range(100):
        node = np.array([x, y, r, g, b])
        ori_con = model.predict_one(perturb_image(node, img))[true_class]

        #if (ori_con < 0.5):
        #    return True

        print("x:{0}, y:{1}, rgb:({2}, {3}, {4}), confidence: {5})".format(x, y, r, g, b, ori_con))
        array = np.empty((0, 32, 32, 3))

        array = np.append(array, np.array((perturb_image(np.array([x, y, bound(r + 1), g, b]), img))), axis=0)
        array = np.append(array, np.array((perturb_image(np.array([x, y, bound(r - 1), g, b]), img))), axis=0)
        array = np.append(array, np.array((perturb_image(np.array([x, y, r, bound(g + 1), b]), img))), axis=0)
        array = np.append(array, np.array((perturb_image(np.array([x, y, r, bound(g - 1), b]), img))), axis=0)
        array = np.append(array, np.array((perturb_image(np.array([x, y, r, g, bound(b + 1)]), img))), axis=0)
        array = np.append(array, np.array((perturb_image(np.array([x, y, r, g, bound(b - 1)]), img))), axis=0)

        processed = model.color_process(array)
        result = model._model.predict(processed, batch_size=6, verbose=0)

        diff_r = result[0][true_class] - result[1][true_class]
        print("diff_r: {0}".format(diff_r))

        diff_g = result[2][true_class] - result[3][true_class]
        print("diff_g: {0}".format(diff_g))

        diff_b = result[4][true_class] - result[5][true_class]
        print("diff_b: {0}".format(diff_b))

        v_r = momentum_rate * v_r - diff_r * rate
        v_g = momentum_rate * v_g - diff_g * rate
        v_b = momentum_rate * v_b - diff_b * rate

        if (abs(diff_r) + abs(diff_g) + abs(diff_b) < 0.0001):
            break

        new_r = bound(r + v_r)
        new_g = bound(g + v_g)
        new_b = bound(b + v_b)

        #if (new_r == r and new_g == g and new_b == b):
        #    break

        r = new_r
        g = new_g
        b = new_b

    helper.plot_image(perturb_image(np.array([x, y, r, g, b]), img))
    print("Attack Confidence:" + str(model.predict_one(perturb_image(np.array([x, y, r, g, b]), img))[true_class]))

def bound(num):
    return int(max(0, min(255,num)))

In [ ]:
model = resnet
image_id = 9

diff_num = pixel_confidence(image_id)
print(diff_num)
x = diff_num // 32
y = diff_num % 32
pixel = np.array([x, y, x_test[image_id][x][y][0], x_test[image_id][x][y][1], x_test[image_id][x][y][2]])

gradient_descent_momentum(image_id, pixel)

In [ ]:
def gradient_ascent_momentum_target(image_id, pixel, target_class):
    img = x_test[image_id]

    x = pixel[0]
    y = pixel[1]
    r = pixel[2]
    g = pixel[3]
    b = pixel[4]

    rate = 1000
    momentum_rate = 0.99

    v_r = 0 #속도
    v_g = 0
    v_b = 0

    for i in range(1000):
        node = np.array([x, y, r, g, b])

        print("x:{0}, y:{1}, rgb:({2}, {3}, {4}), confidence: {5})".format(x, y, r, g, b, model.predict_one(perturb_image(node, img))[target_class]))

        array = np.empty((0, 32, 32, 3))

        array = np.append(array, np.array((perturb_image(np.array([x, y, bound(r + 1), g, b]), img))), axis=0)
        array = np.append(array, np.array((perturb_image(np.array([x, y, bound(r - 1), g, b]), img))), axis=0)
        array = np.append(array, np.array((perturb_image(np.array([x, y, r, bound(g + 1), b]), img))), axis=0)
        array = np.append(array, np.array((perturb_image(np.array([x, y, r, bound(g - 1), b]), img))), axis=0)
        array = np.append(array, np.array((perturb_image(np.array([x, y, r, g, bound(b + 1)]), img))), axis=0)
        array = np.append(array, np.array((perturb_image(np.array([x, y, r, g, bound(b - 1)]), img))), axis=0)

        processed = model.color_process(array)
        result = model._model.predict(processed, batch_size=6, verbose=0)

        diff_r = result[0][target_class] - result[1][target_class]
        #print("diff_r: {0}".format(diff_r))

        diff_g = result[2][target_class] - result[3][target_class]
        #print("diff_g: {0}".format(diff_g))

        diff_b = result[4][target_class] - result[5][target_class]
        #print("diff_b: {0}".format(diff_b))

        v_r = momentum_rate * v_r + diff_r * rate
        v_g = momentum_rate * v_g + diff_g * rate
        v_b = momentum_rate * v_b + diff_b * rate

        new_r = bound(r + v_r)
        new_g = bound(g + v_g)
        new_b = bound(b + v_b)

        if (new_r == r and new_g == g and new_b == b):
            break

        r = new_r
        g = new_g
        b = new_b

    helper.plot_image(perturb_image(np.array([x, y, r, g, b]), img))
    print("Attack Confidence:" + str(model.predict_one(perturb_image(np.array([x, y, r, g, b]), img))[target_class]))

def bound(num):
    return int(max(0, min(255,num)))

In [ ]:
model = resnet
image_id = 384

diff_num = pixel_confidence(image_id)
print(diff_num)
x = diff_num // 32
y = diff_num % 32
pixel = np.array([x, y, x_test[image_id][x][y][0], x_test[image_id][x][y][1], x_test[image_id][x][y][2]])

for i in range(10):
    gradient_ascent_momentum_target(image_id, pixel, i)